In [4]:
import tkinter as tk
import os
import json
import random
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

## LOAD COURSES, DOCTORS, HALLS, DEPARTMENTS ##

with open('courses.json') as f:
    courses = json.load(f)['courses']
    course_name = []
    course_doctor = []
    course_code = []
    course_label = []

    for course in courses:
        course_code.append(course['code'])
        course_name.append(course['name'])
        course_doctor.append(course['doctor'])

with open('doctors.json') as f:
    doctors = json.load(f)['doctors']
    doctor_name = []
    doctor_courses = []
    for row in doctors:
        doctor_name.append(row['name'])
        doctor_courses.append(row['courses'])

with open('halls.json') as f:
    halls = json.load(f)['halls']
    hall_name = []
    hall_capacity = []
    for row in halls:
        hall_name.append(row['name'])
        hall_capacity.append(row['capacity'])

with open('departments.json') as f:
    course_data = json.load(f)

## ENCODING ##

encoder = LabelEncoder()
encoder.fit(doctor_name)
encoded_doctors = encoder.transform(doctor_name)
label_to_doctor = {label: doctor for label, doctor in zip(encoded_doctors, doctor_name)}

encoder2 = LabelEncoder()
encoder2.fit(course_code)
encoded_courses = encoder2.transform(course_code)
label_to_code = {label: code for label, code in zip(encoded_courses, course_code)}

def extract_courses_by_department(course_data):
    courses_by_department = {}
    encoded_dep_courses = []
    for level_data in course_data:
        for department_courses in level_data.values():
            for course in department_courses:
                department_name = course['name']
                courses = [course_info['course'] for course_info in course['courses']]
                encoded_dep_courses.append(encoder2.transform(courses))
                if department_name in courses_by_department:
                    courses_by_department[department_name].extend(courses)
                else:
                    courses_by_department[department_name] = courses
    return courses_by_department, encoded_dep_courses

courses_dep, encoded_dep_courses = extract_courses_by_department(course_data)
encoded_courses_doctors = encoder.transform(course_doctor)
codoc_dict = {course: doctor for course, doctor in zip(encoded_courses, encoded_courses_doctors)}

course_to_dept = {}
for dept in encoded_dep_courses:
    for course in dept:
        course_to_dept[course] = set(dept) - {course}

encoded_dep_items = []
for row in encoded_dep_courses:
    for item in row:
        encoded_dep_items.append(item)

def rand_position():
    num_days = 5
    num_slots = 4
    num_halls = 7
    particle = np.full((num_days, num_slots, num_halls), -1, dtype=int)
    ss = encoded_dep_items.copy()
    slots = [(day, slot, hall) for day in range(num_days) for slot in range(num_slots) for hall in range(num_halls)]
    random.shuffle(slots)
    course_counts = {course: 0 for course in ss}
    dept_courses = {dept: set() for dept in range(len(encoded_dep_courses))}
    for slot in slots:
        day, slot, hall = slot
        if not ss:
            break
        course = random.choice(ss)
        course_dept = [i for i, dept_set in enumerate(encoded_dep_courses) if course in dept_set][0]
        while True:
            if course_counts[course] >= 1:
                break
            if len(encoded_dep_courses[course_dept]) > 2 or course >= 2:
                hall = random.choice([i for i, capacity in enumerate(hall_capacity) if capacity == 500])
            else:
                hall = random.choice([i for i, capacity in enumerate(hall_capacity) if capacity == 200])
            if any(conflict in particle[day, slot] for conflict in course_to_dept[course]) or course in dept_courses[course_dept]:
                day, slot, hall = random.choice(slots)
                continue
            else:
                particle[day][slot][hall] = course
                course_counts[course] += 1
                dept_courses[course_dept].add(course)
                break
    return particle

def constraints_iguess(particle):
    department_sets = [set(dept) for dept in encoded_dep_courses]
    total_doctor_break = 0
    total_department_break = 0
    total_lectures_per_day = [0] * particle.shape[0]
    for day in range(particle.shape[0]):
        for slot in range(particle.shape[1]):
            doctor_break = 0
            department_break = 0
            doctors_in_slot = set()
            courses_in_slot = {i: set() for i in range(len(department_sets))}
            num_lectures = 0
            for hall in range(particle.shape[2]):
                course = particle[day][slot][hall]
                if course == -1:
                    continue
                num_lectures += 1
                if course in codoc_dict:
                    doctor = codoc_dict[course]
                    if doctor in doctors_in_slot:
                        doctor_break += 1
                    doctors_in_slot.add(doctor)
                for i, dept_set in enumerate(department_sets):
                    if course in dept_set:
                        if course in courses_in_slot[i]:
                            department_break += 1
                        courses_in_slot[i].add(course)
            total_doctor_break += doctor_break
            total_department_break += department_break
            total_lectures_per_day[day] += num_lectures
    lectures_constraint = sum(1 for num_lectures in total_lectures_per_day if num_lectures > 4)
    return total_doctor_break, total_department_break, lectures_constraint

class Particle:
    def __init__(self):
        self.position = self.initialize()
        self.velocity = np.zeros_like(self.position)
        self.local_best_position = self.position
        self.local_best_fitness = float('inf')

    def initialize(self):
        return rand_position()

def PSO(objective_fun, swarm_size, maxIter,w,c1,c2):
    swarm_best_position = None
    swarm_best_fitness = float('inf')
    particles = [Particle() for _ in range(swarm_size)]
    all_particle_fitnesses = []
    for itr in range(maxIter):
        iteration_fitnesses = []
        for particle in particles:
            fitness = objective_fun(particle.position)
            iteration_fitnesses.append(fitness)
            if fitness < particle.local_best_fitness:
                particle.local_best_fitness = fitness
                particle.local_best_position = particle.position.copy()
            if fitness < swarm_best_fitness:
                swarm_best_fitness = fitness
                swarm_best_position = particle.position.copy()
        all_particle_fitnesses.append(iteration_fitnesses)
        w = w
        c1 = c1
        c2 = c2
        for particle in particles:
            r1 = np.random.rand(*particle.position.shape)
            r2 = np.random.rand(*particle.position.shape)
            cognitive = c1 * r1 * (particle.local_best_position - particle.position)
            social = c2 * r2 * (swarm_best_position - particle.position)
            particle.velocity = w * particle.velocity + cognitive + social
            particle.velocity = particle.velocity.round().astype(int)
            particle.position += particle.velocity
    return swarm_best_position

def objective_fun(timetable):
    doctor_break, department_break, lectures_constraint = constraints_iguess(timetable)
    penalty = doctor_break + department_break + lectures_constraint
    fitness = penalty
    return fitness

def fill_table(table, timetable):
    for day in range(timetable.shape[0]):  # Iterate over the days
        for slot in range(timetable.shape[1]):  # Iterate over the slots in a day
            course_codes = timetable[day, slot]  # Get the course codes for the day and slot
            course_names = [label_to_code[code] if code != -1 else "" for code in course_codes]  # Get the course names from the codes
            lecturer_names = [course_doctor[code] if code != -1 else "" for code in course_codes]  # Get the lecturer names from the course codes
            values = [f"{course}\n{lecturer}" for course, lecturer in zip(course_names, lecturer_names)]  # Combine course and lecturer names
            value = "\n\n".join(values)  # Join the course and lecturer names with two newline characters
            table[day][slot].config(text=value)  # Update the GUI table cell with the course and lecturer names

def get_pso_params():
    root = tk.Tk()
    root.title("PSO Parameters")
    
    swarm_size_label = tk.Label(root, text="Enter Swarm Size:")
    swarm_size_label.pack()
    swarm_size_entry = tk.Entry(root)
    swarm_size_entry.pack()

    max_iter_label = tk.Label(root, text="Enter Max Iterations:")
    max_iter_label.pack()
    max_iter_entry = tk.Entry(root)
    max_iter_entry.pack()

    w_label = tk.Label(root, text="Enter w:")
    w_label.pack()
    w_entry = tk.Entry(root)
    w_entry.pack()

    c1_label = tk.Label(root, text="Enter c1:")
    c1_label.pack()
    c1_entry = tk.Entry(root)
    c1_entry.pack()

    c2_label = tk.Label(root, text="Enter c2:")
    c2_label.pack()
    c2_entry = tk.Entry(root)
    c2_entry.pack()

    def on_submit():
        global swarm_size, max_iter, w, c1, c2
        swarm_size = int(swarm_size_entry.get())
        max_iter = int(max_iter_entry.get())
        w = float(w_entry.get())
        c1 = float(c1_entry.get())
        c2 = float(c2_entry.get())
        root.destroy()

    submit_button = tk.Button(root, text="Submit", command=on_submit)
    submit_button.pack()

    root.mainloop()

    return swarm_size, max_iter, w, c1, c2

swarm_size, max_iter, w, c1, c2 = get_pso_params()

def create_timetable_gui(day_index, day_name):
    def start_pso():
        best_timetable = PSO(objective_fun, swarm_size, max_iter, w, c1, c2)
        fill_table(table, best_timetable)

    root = tk.Tk()
    root.title(f"Timetable for {day_name}")
    row_labels = ['hall1', 'hall2', 'hall3', 'hall4', 'hall5', 'hall6', 'hall7']
    col_labels = ['8:10', '10:12', '12:2', '2:4']
    light_blue = "#cce6ff"
    light_gray = "#f2f2f2"

    # Labels for row and column headers
    for i, label_text in enumerate(row_labels):
        bg_color = light_blue if i % 2 == 0 else light_gray
        label = tk.Label(root, text=label_text, width=20, height=2, borderwidth=1, relief="solid", bg=bg_color, fg="black", font=("Arial", 12, "bold"))
        label.grid(row=i+1, column=0, padx=5, pady=5, sticky="nsew")
    for i, label_text in enumerate(col_labels):
        label = tk.Label(root, text=label_text, width=20, height=2, borderwidth=1, relief="solid", bg=light_blue, fg="black", font=("Arial", 12, "bold"))
        label.grid(row=0, column=i+1, padx=5, pady=5, sticky="nsew")

    # Table for displaying timetable
    table = [[None] * 4 for _ in range(7)]
    for row in range(7):
        for col in range(4):
            bg_color = light_blue if row % 2 == 0 else light_gray
            label = tk.Label(root, text="", width=20, height=2, borderwidth=1, relief="solid", bg=bg_color, fg="black", font=("Arial", 12))
            label.grid(row=row+1, column=col+1, padx=5, pady=5, sticky="nsew")
            table[row][col] = label

    start_pso()
    root.mainloop()

days = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday']
for day_index, day_name in enumerate(days):
    create_timetable_gui(day_index, day_name)